# How-to train a model on Azure ML

This notebook takes you through the steps of training a model on Azure ML for The Ocean Cleanup. We train the models through Azure ML to provide us with a good registration of all performed tests, so that we can see why and how a model was created.

When the result of a training run is satisfactory, a model can be registered from there, from which point we can deploy it.

There are a few concepts to know about first:

- Workspace: The entire AzureML environment you are working in. The Workspace contains all the other elements.
- Experiment: A collection of Runs (see below). A logical container for training a model with different parameters to determine the best.
- Run: A single train/test run of a model. These are tied to an experiment. If you want to train the same model with different parameters, so you can compare them, these are different runs under the same experiment.
- Environment: The code environment used by your code. This contains things like the required Python packages. Multiple options exist here - from just using your local environment to completely curated environments directly from Azure.
- Dataset: A single dataset as registered in the AzureML workspace.

With that out of the way, lets dive right in. Looking at these components, our first step will be to get the correct Workspace:

In [1]:
from azurewrapper.workspace import get_workspace

subscription_id = "29d66431-a7ce-4709-93f7-3bdb01a243b3"
resource_group = "ExperimentationJayke"
workspace_name = "ExperimentationJayke"

workspace = get_workspace(subscription_id, resource_group, workspace_name)

## Create experiment

Now that we have a workspace available, we need to create an experiment. As describe above, an experiment will be the container for multiple runs, in which we can train and compare the model using different parameters.

The experiment needs a name. Use something that is descriptive and clear to anyone seeing this.

In [2]:
from azurewrapper.train import create_experiment
experiment = create_experiment(workspace, "model-yolov5-v-1-0")

## Create or select compute target

We want to train our model on a GPU cluster on AzureML. Lets create one (or load an existing one).

In [3]:
from azurewrapper.compute import get_compute

compute_target = get_compute(workspace, "gpu-cluster", vm_size='STANDARD_NC6', max_nodes=4)

Found existing compute target


## Create the environment

We will now need to create an environment. In this case, we build the enviroment based of the Azure docker Tensorflow image, but with our own pip requirements.

In [4]:
from azureml.core import Environment
from azurewrapper.environment import get_environment


#environment = get_environment(workspace, 'AzureML-TensorFlow-2.2-GPU')
#environment.save_to_directory(path='deps.yml')
# environment = Environment.from_pip_requirements(name="pytorch-exp", file_path="./examples/yolov5/yolov5/requirements.txt")
# environment.docker.enabled = True
# environment.docker.base_image = "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04:20201019.v1"

environment = Environment.from_pip_requirements(
    "pytorch-exp-custom-docker",
    file_path="./examples/yolov5/yolov5/requirements.txt"
)

# Load from dockerfile
environment.docker.enabled = True
environment.docker.base_image = None
environment.docker.base_dockerfile = "./examples/yolov5/Dockerfile"

## Prepare model wraper

Now it's time to perform our first Run of the experiment. However, before we can do this, we will need a wrapper around our model. This wrapper needs to do a few things:

- Initialize and train the model with:
  - The desired parameters
  - The desired data
- Evaluate the performance of the trained model
- Register the parameters and the performance in the Run object
- Add the generated model artifacts to the Run object

There is skeleton code for this available: `skeleton_files/train.py`. In this file you fill in what parameters you expect, you create and train and evaluate the model using these parameters and the loaded in dataset(s), and you register the results and the created artifacts with the Run.

For this how-to, we will use the example provided in `examples/yolov5/train.py`. This is an implementation of the file mentioned above. It accepts x parameters: `LR_INIT`, `LR_END`, `WARMUP_EPOCHS` and `EPOCHS`, and requires `yolov5 weights` as dataset.

### Changes to yolo code

- Added AzureML files to `yolov5/requirements.txt`

## Run the experiment

Now we need to create and run the experiment. First, we fetch the desired datasets, and combine these into train- and test sets. Note that we can provide multiple sets for both training and testing. Also note that each set consists of both a label and an image dataset.

In [5]:
from azureml.core import Dataset

train_images = Dataset.get_by_name(workspace, name="campaign-26-10-2020_images")
train_labels = Dataset.get_by_name(workspace, name="campaign-26-10-2020_labels")
test_images = Dataset.get_by_name(workspace, name="campaign-22-10-2020_images")
test_labels = Dataset.get_by_name(workspace, name="campaign-22-10-2020_labels")
trainsets = [
    (train_labels, train_images),
    (test_labels, test_images)
]
testsets = [
    (test_labels, test_images)
]

We now have everything we need to perform the run. Lets do so!

In [31]:
from azurewrapper.train import perform_run
from azureml.core.runconfig import TensorflowConfiguration

weights = Dataset.get_by_name(workspace, name="YOLOV5")

run = perform_run(experiment, 'train.py', 'examples/yolov5', environment=environment,
                  trainsets=trainsets, testsets=testsets, compute_target=compute_target,
                  parameters={
                      'weights': weights.as_named_input(f'weights').as_mount(),
                  })
run.wait_for_completion(show_output=True)

RunId: model-yolov5-v-1-0_1604590268_d1893f27
Web View: https://ml.azure.com/experiments/model-yolov5-v-1-0/runs/model-yolov5-v-1-0_1604590268_d1893f27?wsid=/subscriptions/29d66431-a7ce-4709-93f7-3bdb01a243b3/resourcegroups/ExperimentationJayke/workspaces/ExperimentationJayke

Streaming azureml-logs/55_azureml-execution-tvmps_f9541cb19714667ef7d3a99fc4579027311bf611496ab7b506844081e15d0bd9_d.txt

2020-11-05T15:36:46Z Starting output-watcher...
2020-11-05T15:36:46Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2020-11-05T15:36:47Z Executing 'Copy ACR Details file' on 10.0.0.5
2020-11-05T15:36:47Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_4a19f9495f68b1b64d3ab073b622096f
171857c49d0f: Pulling fs layer
419640447d26: Pulling fs layer
61e52f862619: Pulling fs layer
c118dad7e37a: Pulling fs layer
2e36372995f9: Pulling fs layer
0b8e00a4ba4e: Pulling fs layer
b3026b4f2581: P

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='experimentatio5321470455.blob.core.windows.net', port=443): Read timed out. (read timeout=120)",)': /azureml/ExperimentRun/dcid.model-yolov5-v-1-0_1604590268_d1893f27/azureml-logs/70_driver_log.txt?sv=2019-02-02&sr=b&sig=LKuZdf%2FAF9ObGP0h2lWa3QGAHVuXraivKyCp93zJVQo%3D&st=2020-11-05T15%3A30%3A10Z&se=2020-11-05T23%3A40%3A10Z&sp=r


2020/11/05 15:39:26 logger.go:297: Attempt 1 of http call to http://10.0.0.5:16384/sendlogstoartifacts/info
2020/11/05 15:39:26 logger.go:297: Attempt 1 of http call to http://10.0.0.5:16384/sendlogstoartifacts/status
[2020-11-05T15:39:27.708174] Entering context manager injector.
[context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'Dataset:context_managers.Datasets', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['train.py', '--train_sets', '0046c288-9c27-407a-afb5-d0439c374cbe', 'DatasetConsumptionConfig:train_images_0', 'bc619fbb-e949-42f5-9259-41301434c740', 'DatasetConsumptionConfig:train_images_1', '--test_sets', 'bc619fbb-e949-42f5-9259-41301434c740', 'DatasetConsumptionConfig:test_images_0', '--weights', 'DatasetConsumptionConfig:weights'])
Initialize DatasetContextManager.
Starting the daemon thread to refresh tokens in background for process with


Streaming azureml-logs/75_job_post-tvmps_f9541cb19714667ef7d3a99fc4579027311bf611496ab7b506844081e15d0bd9_d.txt

Entering job release. Current time:2020-11-05T15:41:47.598824
Starting job release. Current time:2020-11-05T15:41:48.602687
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 1091
[2020-11-05T15:41:48.603684] job release stage : upload_datastore starting...
[{}] job release stage : start importing azureml.history._tracking in run_history_release.
[2020-11-05T15:41:48.604204] job release stage : copy_batchai_cached_logs starting...
[2020-11-05T15:41:48.604250] job release stage : copy_batchai_cached_logs completed...
[2020-11-05T15:41:48.604326] job release stage : execute_job_release starting...
[2020-11-05T15:41:48.612679] Entering context manager injector.
[2020-11-05T15:41:48.613879] job release stage : upload_datastore completed...
[2020-11-05T15:41:48.838028] job release stage : sen

{'runId': 'model-yolov5-v-1-0_1604590268_d1893f27',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-05T15:36:45.795722Z',
 'endTimeUtc': '2020-11-05T15:41:59.131637Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '57ed6ce5-c8f0-446a-8e3f-c3900140217f',
  'azureml.git.repository_uri': 'git@github.com:TheOceanCleanup/AIDataPipeLine.git',
  'mlflow.source.git.repoURL': 'git@github.com:TheOceanCleanup/AIDataPipeLine.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'bdc118d3fd32a00e63b52298b3d8a3afc275308a',
  'mlflow.source.git.commit': 'bdc118d3fd32a00e63b52298b3d8a3afc275308a',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '49fac13f-e0d8-4622-a4ac-f3634ec5e35b'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'test_images_0', 'm